# Creating a basic chat experience with kernel arguments

In this example, we show how you can build a simple chat bot by sending and updating the kernel arguments with your requests.

We introduce the Kernel Arguments object which in this demo functions similarly as a key-value store that you can use when running the kernel.

The chat history is local (i.e. in your computer's RAM) and not persisted anywhere beyond the life of this Jupyter session.

In future examples, we will show how to persist the chat history on disk so that you can bring it into your applications.

In this chat scenario, as the user talks back and forth with the bot, the chat context gets populated with the history of the conversation. During each new run of the kernel, the kernel arguments and chat history can provide the AI with its variables' content.


In [1]:
# Note: if using a virtual environment, do not run this cell
%pip install -U semantic-kernel
from semantic_kernel import __version__

__version__

Note: you may need to restart the kernel to use updated packages.


'1.9.0'

Initial configuration for the notebook to run properly.

In [2]:
# Make sure paths are correct for the imports

import os
import sys

# notebook_dir = os.path.abspath("")
# parent_dir = os.path.dirname(notebook_dir)
# grandparent_dir = os.path.dirname(parent_dir)


# sys.path.append(grandparent_dir)

### Configuring the Kernel

Let's get started with the necessary configuration to run Semantic Kernel. For Notebooks, we require a `.env` file with the proper settings for the model you use. Create a new file named `.env` and place it in this directory. Copy the contents of the `.env.example` file from this directory and paste it into the `.env` file that you just created.

**NOTE: Please make sure to include `GLOBAL_LLM_SERVICE` set to either OpenAI, AzureOpenAI, or HuggingFace in your .env file. If this setting is not included, the Service will default to AzureOpenAI.**

#### Using Local LLM Ollama


```
GLOBAL_LLM_SERVICE="Ollama"
OLLAMA_API_BASE_URL="http://localhost:11434"
OLLAMA_MODEL_NAME="llama3"
OLLAMA_EMBED_MODEL_NAME="llama3"
OLLAMA_TIMEOUT_SECONDS=120

```

For more advanced configuration, please follow the steps outlined in the [setup guide](./CONFIGURING_THE_KERNEL.md).

We will load our settings and get the LLM service to use for the notebook.

In [22]:
from semantic_kernel.kernel_pydantic import KernelBaseSettings
from typing import Optional

class ServiceSettings(KernelBaseSettings):
    """The Learn Resources Service Settings.

    The settings are first loaded from environment variables. If the
    environment variables are not found, the settings can be loaded from a .env file with the
    encoding 'utf-8' as default or the specific encoding. If the settings are not found in the
    .env file, the settings are ignored; however, validation will fail alerting that the settings
    are missing.

    Args:
        global_llm_service (str | None): The LLM service to use for the samples, either "OpenAI", "AzureOpenAI", or "Ollama"
            If not provided, defaults to "AzureOpenAI".
    """

    global_llm_service: Optional[str] = None
    ollama_api_base_url: Optional[str] = None
    ollama_model_name: Optional[str] = None
    ollama_embed_model_name: Optional[str] = None
    ollama_timeout_seconds: Optional[int] = None
    ollama_chat_model_name: Optional[str] = None
    ollama_text_model_name: Optional[str] = None
    ollama_headers: Optional[str] = None

In [23]:
from enum import Enum
class Service(Enum):
    OpenAI = "openai"
    AzureOpenAI = "azureopenai"
    HuggingFace = "huggingface"
    Ollama = "ollama"

service_settings = ServiceSettings.create()

In [24]:
from semantic_kernel.connectors.ai.ollama.services.ollama_text_completion import OllamaTextCompletion
from semantic_kernel.connectors.ai.ollama.services.ollama_chat_completion import OllamaChatCompletion


selectedService = Service.AzureOpenAI if service_settings.global_llm_service is None else Service(service_settings.global_llm_service.lower())
print(f"Using service type: {selectedService}")

if selectedService == Service.Ollama:
    ollama_text_service = OllamaTextCompletion(
        ai_model_id=service_settings.ollama_model_name,
        host=service_settings.ollama_api_base_url
    )

    ollama_chat_service = OllamaChatCompletion(
        ai_model_id=service_settings.ollama_model_name,
        host=service_settings.ollama_api_base_url
    )

Using service type: Service.Ollama


In [27]:
from semantic_kernel import Kernel

kernel = Kernel()

service_id = None
if selectedService == Service.Ollama:
    kernel.add_service(ollama_text_service)
    # kernel.add_service(ollama_chat_service)



# if selectedService == Service.OpenAI:
#     from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

#     service_id = "default"
#     kernel.add_service(
#         OpenAIChatCompletion(
#             service_id=service_id,
#         ),
#     )
# elif selectedService == Service.AzureOpenAI:
#     from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

#     service_id = "default"
#     kernel.add_service(
#         AzureChatCompletion(
#             service_id=service_id,
#         ),
#     )

Let's define a prompt outlining a dialogue chat bot.


In [28]:
prompt = """
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$user_input}}
ChatBot: """

Register your semantic function


In [29]:
from semantic_kernel.connectors.ai.ollama.ollama_prompt_execution_settings import OllamaPromptExecutionSettings
from semantic_kernel.prompt_template import PromptTemplateConfig
from semantic_kernel.prompt_template.input_variable import InputVariable

if selectedService == Service.Ollama:
    execution_settings = OllamaPromptExecutionSettings(
        ai_model_id=service_settings.ollama_model_name,
        max_tokens=2000,
        temperature=0.7,
    )

prompt_template_config = PromptTemplateConfig(
    template=prompt,
    name="chat",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="user_input", description="The user input", is_required=True),
        InputVariable(name="history", description="The conversation history", is_required=True),
    ],
    execution_settings=execution_settings,
)

chat_function = kernel.add_function(
    function_name="chat",
    plugin_name="chatPlugin",
    prompt_template_config=prompt_template_config,
)

In [30]:
from semantic_kernel.contents import ChatHistory

chat_history = ChatHistory()
chat_history.add_system_message("You are a helpful chatbot who is good about giving book recommendations.")

Initialize the Kernel Arguments


In [31]:
from semantic_kernel.functions import KernelArguments

arguments = KernelArguments(user_input="Hi, I'm looking for book suggestions", history=chat_history)

Chat with the Bot


In [32]:
response = await kernel.invoke(chat_function, arguments)
print(response)

Hi there! I'd be happy to help you with some book recommendations!

To get started, can you please tell me a bit more about what you're in the mood for? Are you looking for something:

* Specific genre (e.g. fantasy, mystery, romance)?
* In a particular age range (e.g. children's, young adult, adult)?
* Based on a certain theme or setting?
* By a specific author or recommended by someone else?

Let me know and I'll do my best to suggest some great books for you!


Update the history with the output


In [33]:
chat_history.add_assistant_message(str(response))

Keep Chatting!


In [34]:
async def chat(input_text: str) -> None:
    # Save new message in the context variables
    print(f"User: {input_text}")

    # Process the user message and get an answer
    answer = await kernel.invoke(chat_function, KernelArguments(user_input=input_text, history=chat_history))

    # Show the response
    print(f"ChatBot: {answer}")

    chat_history.add_user_message(input_text)
    chat_history.add_assistant_message(str(answer))

In [35]:
await chat("I love history and philosophy, I'd like to learn something new about Greece, any suggestion?")

User: I love history and philosophy, I'd like to learn something new about Greece, any suggestion?
ChatBot: What a fascinating combination of interests!

For someone who loves history and philosophy, I think you might enjoy "The Greeks" by H.D. Kitto. This classic book is a comprehensive introduction to ancient Greek culture, covering their myths, literature, art, architecture, and politics. It's written in an engaging and accessible style, making it perfect for those new to the subject.

Another great option could be "The Odyssey" by Homer (translated by Robert Fagles). As one of the most influential works of Western literature, this epic poem is a must-read for anyone interested in ancient Greece. It tells the story of Odysseus' journey home after the Trojan War and explores themes of identity, morality, and the human condition.

If you're looking for something more philosophical, I'd recommend "The Republic" by Plato (translated by Benjamin Jowett). This foundational work of Western

In [37]:
await chat("that sounds interesting, what is it about?")

User: that sounds interesting, what is it about?
ChatBot: Let me give you a brief summary of each book:

**"The Greeks" by H.D. Kitto**: This book is an introduction to ancient Greek culture, covering their mythology, literature, art, architecture, and politics. It's a comprehensive overview that will give you a sense of what Greece was like during its Golden Age. You'll learn about the gods and goddesses, famous philosophers, and cultural achievements that have had a lasting impact on Western civilization.

**"The Odyssey" by Homer (translated by Robert Fagles)**: This epic poem tells the story of Odysseus' journey home after the Trojan War. You'll follow him as he faces various challenges and encounters with mythical creatures, gods, and goddesses. The book explores themes such as identity, morality, and the human condition, making it a great choice for those interested in philosophy.

**"The Republic" by Plato (translated by Benjamin Jowett)**: This foundational work of Western phil

In [ ]:
await chat("if I read that book, what exactly will I learn about Greek history?")

In [ ]:
await chat("could you list some more books I could read about this topic?")

After chatting for a while, we have built a growing history, which we are attaching to each prompt and which contains the full conversation. Let's take a look!


In [36]:
print(chat_history)

<chat_history><message role="system"><text>You are a helpful chatbot who is good about giving book recommendations.</text></message><message role="assistant"><text>Hi there! I'd be happy to help you with some book recommendations!

To get started, can you please tell me a bit more about what you're in the mood for? Are you looking for something:

* Specific genre (e.g. fantasy, mystery, romance)?
* In a particular age range (e.g. children's, young adult, adult)?
* Based on a certain theme or setting?
* By a specific author or recommended by someone else?

Let me know and I'll do my best to suggest some great books for you!</text></message><message role="user"><text>I love history and philosophy, I'd like to learn something new about Greece, any suggestion?</text></message><message role="assistant"><text>What a fascinating combination of interests!

For someone who loves history and philosophy, I think you might enjoy "The Greeks" by H.D. Kitto. This classic book is a comprehensive intr